### 협업필터링(Collaborative filtering)
- 사용자들의 데이터를 기반으로 대상(target)과 유사한(높은 유사도를 갖는) 사용자들의 데이터를 분석하여 추천함
- 추천을 하는 데이터는 회사 내부에 모아져있는 상태여야 시작이 가능함(cold start가 불가능)
- 나와 비슷한 사람으로부터 추천을 받는 것이 더 좋다는 원리
- 개인에게 개인화된 추천이 가능함

In [67]:
import pandas as pd
import numpy as np

In [115]:
url = 'https://raw.githubusercontent.com/luxdolorosa/data_set/master/movie/ratings_UTF8.csv'
ratings = pd.read_csv(url, encoding='utf-8', sep='\t', index_col=0)

In [69]:
ratings.head()

,name,title,rating
no,,,
1,최트루,택시운전사,3.0
2,최트루,신과함께,4.0
3,최트루,공조,3.5
4,최트루,스파이더맨,5.0
5,최트루,범죄도시,2.0


# 유사도 구하기
- 1. 리뷰어 별로 각 영화에 몇 점을 주었는가?

In [91]:
movie_ratings = pd.pivot(ratings, index='title', columns='name', values='rating')

- 2. 사용자 사이의 유사도 계산

In [93]:
sim_user = movie_ratings.corr().reset_index()
sim_user

name,name,조쌤,지리산,최트루,코리아,포세이돈,헤르메스
0,조쌤,1.000000,0.151247,0.440589,0.600081,0.367410,-0.563602
1,지리산,0.151247,1.000000,-0.184000,0.616259,0.754726,0.316228
2,최트루,0.440589,-0.184000,1.000000,0.464565,0.388354,0.154919
3,코리아,0.600081,0.616259,0.464565,1.000000,0.633920,-0.075593
4,포세이돈,0.367410,0.754726,0.388354,0.633920,1.000000,0.946729
5,헤르메스,-0.563602,0.316228,0.154919,-0.075593,0.946729,1.000000


# 헤르메스한테 추천할 영화 정보를 구하기
- 헤르메스가 평가하지 않은 영화를 골라내기

In [73]:
tmp_title = movie_ratings['헤르메스']
tmp_title.isnull()
na_title = tmp_title[tmp_title.isnull()].reset_index()

In [74]:
na_list = list(na_title['title'])
na_list

['군함도', '신과함께']

In [75]:
# 원본데이터에서 헤르메스가 평가하지 않은 영화들(군함도, 신과함께)의 다른 평점을 조회
rating_t = ratings[ratings.apply(lambda x: x.title in na_list, axis=1)]

In [76]:
sim_user.loc[:, ['name','헤르메스']]
temp_ratings = pd.merge(rating_t, sim_user.loc[:, ['name','헤르메스']], on='name')

In [77]:
temp_ratings.sort_values(['헤르메스', 'rating'], ascending=False)
#  .iloc[0,1]
# 헤르메스에게는 유사도가 높은 포세이돈이 추천하는 신과함께가 가장 재미있을 영화이다.

,name,title,rating,헤르메스
8,포세이돈,신과함께,5.0,0.946729
9,포세이돈,군함도,3.0,0.946729
2,지리산,신과함께,5.0,0.316228
3,지리산,군함도,3.5,0.316228
0,최트루,신과함께,4.0,0.154919
1,최트루,군함도,4.0,0.154919
6,코리아,신과함께,5.0,-0.075593
7,코리아,군함도,4.0,-0.075593
4,조쌤,신과함께,5.0,-0.563602
5,조쌤,군함도,2.0,-0.563602


# 검증 및 데이터를 변경
- 헤르메스가 안 본 영화 중 신과함께를 공조로 변경
- 변경된 신과함께 평점을 5점으로 변경

In [116]:
ratings.loc[(ratings.title == '공조') & (ratings.name == '헤르메스'), :] = ['헤르메스','신과함께',5.0]

In [117]:
movie_ratings = pd.pivot(ratings, index='title', columns='name', values='rating')

In [118]:
sim_user = movie_ratings.corr().reset_index()
tmp_title = movie_ratings['헤르메스']
na_title = tmp_title[tmp_title.isnull()].reset_index()
na_list = list(na_title['title'])
rating_t = ratings[ratings.apply(lambda x: x.title in na_list, axis=1)]
tmp_ratings = pd.merge(rating_t, sim_user.loc[:, ['name','헤르메스']], on='name')

In [99]:
# 유사도가 가장 높은 1인으로 할 경우에는 포세이돈이 가장 높게 평가한 군함도를 추천하면 되지만
# 다른 유사도가 높은 유저가 많은 경우에는 1인에게 받는 것보다는
# 다른 유저들의 데이터도 같이 반영되는게 일반적으로 더 좋은 성능을 보인다.

In [119]:
# 헤르메스라는 컬럼은 헤르메스와 다른 유저와의 유사도이므로 컬럼명을 similarity로 변경하기
tmp_ratings.rename(columns={'헤르메스':'similarity'}, inplace=True)

In [120]:
tmp_ratings['sim_rating'] = tmp_ratings.rating * tmp_ratings.similarity
tmp_ratings.sort_values('sim_rating', ascending=False).iloc[0,1]
# .iloc[0,1]

'공조'

In [122]:
# 헤르메스의 예상 점수를 구할 수도 있음
rate_temp = tmp_ratings.groupby('title').sum()
rate_temp.sim_rating / rate_temp.similarity

title
공조     3.453851
군함도    3.252310
dtype: float64